# 第十一章 標準程式庫 (Part 1: 基礎I/O和時間處理)

Go 的標準程式庫提供了豐富且強大的功能，涵蓋了從基本的 I/O 操作到複雜的網路通訊。標準程式庫的設計遵循 Go 的設計哲學：簡潔、高效且易於使用。

本章將深入探討標準程式庫中最重要的幾個套件，這些套件是 Go 程式開發的基石。我們將學習如何有效地使用這些工具來構建強大的應用程式。

Part 1 將專注於基礎的 I/O 操作和時間處理功能。

## io 與朋友

`io` 套件定義了 Go 中最基本的 I/O 介面，這些介面在整個標準程式庫中被廣泛使用。理解這些介面對於掌握 Go 的 I/O 操作至關重要。

### 核心介面：
- `io.Reader`: 定義了 `Read([]byte) (int, error)` 方法
- `io.Writer`: 定義了 `Write([]byte) (int, error)` 方法
- `io.Closer`: 定義了 `Close() error` 方法
- `io.Seeker`: 定義了 `Seek(int64, int) (int64, error)` 方法

### 組合介面：
- `io.ReadWriter`: 組合了 Reader 和 Writer
- `io.ReadCloser`: 組合了 Reader 和 Closer
- `io.WriteCloser`: 組合了 Writer 和 Closer

這些介面的強大之處在於它們的組合性和一致性，讓不同的 I/O 操作可以用統一的方式處理。

In [ ]:
package main

import (
    "fmt"
    "io"
    "strings"
)

/* io.Reader 基本使用 */
func basicReaderExample() {
    // strings.Reader 實作了 io.Reader 介面
    reader := strings.NewReader("Hello, World!")
    
    // 建立緩衝區來讀取資料
    buffer := make([]byte, 5)
    
    for {
        n, err := reader.Read(buffer)
        if err == io.EOF {
            fmt.Println("讀取完成")
            break
        }
        if err != nil {
            fmt.Printf("讀取錯誤: %v\n", err)
            break
        }
        
        fmt.Printf("讀取了 %d 個位元組: %q\n", n, string(buffer[:n]))
    }
    // 輸出:
    // 讀取了 5 個位元組: "Hello"
    // 讀取了 5 個位元組: ", Wor"
    // 讀取了 3 個位元組: "ld!"
    // 讀取完成
}

func main() {
    basicReaderExample()
}

In [ ]:
package main

import (
    "bytes"
    "fmt"
    "io"
)

/* io.Writer 基本使用 */
func basicWriterExample() {
    // bytes.Buffer 實作了 io.Writer 介面
    var buffer bytes.Buffer
    
    // 寫入資料
    data := []string{"Hello", ", ", "World", "!"}
    
    for i, str := range data {
        n, err := buffer.Write([]byte(str))
        if err != nil {
            fmt.Printf("寫入錯誤: %v\n", err)
            return
        }
        fmt.Printf("第 %d 次寫入了 %d 個位元組: %q\n", i+1, n, str)
    }
    
    fmt.Printf("緩衝區內容: %q\n", buffer.String())
    fmt.Printf("緩衝區長度: %d\n", buffer.Len())
    
    // 輸出:
    // 第 1 次寫入了 5 個位元組: "Hello"
    // 第 2 次寫入了 2 個位元組: ", "
    // 第 3 次寫入了 5 個位元組: "World"
    // 第 4 次寫入了 1 個位元組: "!"
    // 緩衝區內容: "Hello, World!"
    // 緩衝區長度: 13
}

func main() {
    basicWriterExample()
}

In [ ]:
package main

import (
    "fmt"
    "io"
    "strings"
)

/* io.Copy 函式的強大功能 */
func ioCopyExample() {
    // 建立一個 Reader
    source := strings.NewReader("This is source data for copying.")
    
    // 建立一個 Writer
    var destination strings.Builder
    
    // 使用 io.Copy 將資料從 Reader 複製到 Writer
    bytesWritten, err := io.Copy(&destination, source)
    if err != nil {
        fmt.Printf("複製錯誤: %v\n", err)
        return
    }
    
    fmt.Printf("成功複製 %d 個位元組\n", bytesWritten)
    fmt.Printf("目標內容: %q\n", destination.String())
    
    // 輸出:
    // 成功複製 33 個位元組
    // 目標內容: "This is source data for copying."
}

func main() {
    ioCopyExample()
}

In [ ]:
package main

import (
    "fmt"
    "io"
    "strings"
)

/* 使用 io.ReadAll 和其他實用函式 */
func ioUtilitiesExample() {
    data := "Go is a great programming language!"
    reader := strings.NewReader(data)
    
    // 使用 io.ReadAll 一次讀取所有資料
    fmt.Println("使用 io.ReadAll:")
    allData, err := io.ReadAll(reader)
    if err != nil {
        fmt.Printf("讀取錯誤: %v\n", err)
        return
    }
    fmt.Printf("讀取到: %q\n", string(allData))
    
    // 重新建立 reader 因為上面已經讀完了
    reader2 := strings.NewReader(data)
    
    // 使用 io.LimitReader 限制讀取數量
    fmt.Println("\n使用 io.LimitReader (限制 15 個位元組):")
    limitedReader := io.LimitReader(reader2, 15)
    
    limitedData, err := io.ReadAll(limitedReader)
    if err != nil {
        fmt.Printf("讀取錯誤: %v\n", err)
        return
    }
    fmt.Printf("限制讀取到: %q\n", string(limitedData))
    
    // 輸出:
    // 使用 io.ReadAll:
    // 讀取到: "Go is a great programming language!"
    // 
    // 使用 io.LimitReader (限制 15 個位元組):
    // 限制讀取到: "Go is a great p"
}

func main() {
    ioUtilitiesExample()
}

In [ ]:
package main

import (
    "fmt"
    "io"
    "strings"
)

/* io.MultiReader 和 io.MultiWriter 的使用 */
func multiReaderWriterExample() {
    // 建立多個 Reader
    reader1 := strings.NewReader("First part, ")
    reader2 := strings.NewReader("second part, ")
    reader3 := strings.NewReader("third part.")
    
    // 使用 io.MultiReader 組合多個 Reader
    multiReader := io.MultiReader(reader1, reader2, reader3)
    
    fmt.Println("使用 MultiReader 讀取:")
    data, err := io.ReadAll(multiReader)
    if err != nil {
        fmt.Printf("讀取錯誤: %v\n", err)
        return
    }
    fmt.Printf("組合讀取結果: %q\n", string(data))
    
    // 建立多個 Writer 目標
    var writer1, writer2, writer3 strings.Builder
    
    // 使用 io.MultiWriter 同時寫入多個目標
    multiWriter := io.MultiWriter(&writer1, &writer2, &writer3)
    
    fmt.Println("\n使用 MultiWriter 寫入:")
    message := "This message goes to all writers!"
    n, err := multiWriter.Write([]byte(message))
    if err != nil {
        fmt.Printf("寫入錯誤: %v\n", err)
        return
    }
    
    fmt.Printf("寫入了 %d 個位元組到所有 writers\n", n)
    fmt.Printf("Writer1 內容: %q\n", writer1.String())
    fmt.Printf("Writer2 內容: %q\n", writer2.String())
    fmt.Printf("Writer3 內容: %q\n", writer3.String())
    
    // 輸出:
    // 使用 MultiReader 讀取:
    // 組合讀取結果: "First part, second part, third part."
    // 
    // 使用 MultiWriter 寫入:
    // 寫入了 33 個位元組到所有 writers
    // Writer1 內容: "This message goes to all writers!"
    // Writer2 內容: "This message goes to all writers!"
    // Writer3 內容: "This message goes to all writers!"
}

func main() {
    multiReaderWriterExample()
}

In [ ]:
package main

import (
    "bufio"
    "fmt"
    "strings"
)

/* bufio 套件的緩衝I/O操作 */
func bufioExample() {
    text := `第一行
第二行
第三行
第四行`
    
    reader := strings.NewReader(text)
    
    // 建立緩衝讀取器
    bufReader := bufio.NewReader(reader)
    
    fmt.Println("使用 bufio.Scanner 逐行讀取:")
    
    // 重新建立 scanner
    reader2 := strings.NewReader(text)
    scanner := bufio.NewScanner(reader2)
    
    lineNum := 1
    for scanner.Scan() {
        line := scanner.Text()
        fmt.Printf("第 %d 行: %q\n", lineNum, line)
        lineNum++
    }
    
    if err := scanner.Err(); err != nil {
        fmt.Printf("掃描錯誤: %v\n", err)
    }
    
    fmt.Println("\n使用 bufio.Reader 讀取單行:")
    reader3 := strings.NewReader("Hello, World!\nSecond line")
    bufReader2 := bufio.NewReader(reader3)
    
    line, err := bufReader2.ReadString('\n')
    if err != nil {
        fmt.Printf("讀取錯誤: %v\n", err)
    } else {
        fmt.Printf("讀取到的行: %q\n", line)
    }
    
    // 輸出:
    // 使用 bufio.Scanner 逐行讀取:
    // 第 1 行: "第一行"
    // 第 2 行: "第二行"
    // 第 3 行: "第三行"
    // 第 4 行: "第四行"
    // 
    // 使用 bufio.Reader 讀取單行:
    // 讀取到的行: "Hello, World!\n"
}

func main() {
    bufioExample()
}

## time

`time` 套件提供了 Go 中處理時間和日期的功能。它包含了時間的表示、格式化、解析、計算等各種操作。Go 的時間處理有一些獨特的特點，特別是時間格式化使用參考時間的方式。

### 核心概念：
- `time.Time`: 表示一個時間點
- `time.Duration`: 表示兩個時間點之間的間隔
- `time.Location`: 表示時區
- Go 的參考時間: `Mon Jan 2 15:04:05 MST 2006` (Unix 時間戳: 1136239445)

### 重要功能：
- 時間的建立、格式化和解析
- 時間計算和比較
- 計時器和定時器
- 時區處理

In [ ]:
package main

import (
    "fmt"
    "time"
)

/* time.Time 基本使用 */
func basicTimeExample() {
    // 獲取當前時間
    now := time.Now()
    fmt.Printf("當前時間: %v\n", now)
    fmt.Printf("年: %d, 月: %d, 日: %d\n", now.Year(), now.Month(), now.Day())
    fmt.Printf("時: %d, 分: %d, 秒: %d\n", now.Hour(), now.Minute(), now.Second())
    fmt.Printf("星期: %v\n", now.Weekday())
    
    // 建立特定時間
    specificTime := time.Date(2023, time.December, 25, 15, 30, 0, 0, time.UTC)
    fmt.Printf("\n特定時間: %v\n", specificTime)
    fmt.Printf("Unix 時間戳: %d\n", specificTime.Unix())
    fmt.Printf("毫秒時間戳: %d\n", specificTime.UnixMilli())
    
    // 從 Unix 時間戳建立時間
    timestamp := int64(1703520600) // 2023-12-25 15:30:00 UTC
    fromUnix := time.Unix(timestamp, 0)
    fmt.Printf("\n從時間戳建立: %v\n", fromUnix)
    
    // 時間比較
    fmt.Printf("\n時間比較:\n")
    fmt.Printf("now.After(specificTime): %t\n", now.After(specificTime))
    fmt.Printf("now.Before(specificTime): %t\n", now.Before(specificTime))
    fmt.Printf("specificTime.Equal(fromUnix): %t\n", specificTime.Equal(fromUnix))
}

func main() {
    basicTimeExample()
}

In [ ]:
package main

import (
    "fmt"
    "time"
)

/* time.Duration 使用和時間計算 */
func durationExample() {
    // 不同的時間長度表示
    fmt.Println("時間長度常數:")
    fmt.Printf("1 毫秒: %v\n", time.Millisecond)
    fmt.Printf("1 秒: %v\n", time.Second)
    fmt.Printf("1 分鐘: %v\n", time.Minute)
    fmt.Printf("1 小時: %v\n", time.Hour)
    
    // 自訂時間長度
    customDuration := 2*time.Hour + 30*time.Minute + 45*time.Second
    fmt.Printf("\n自訂時間長度: %v\n", customDuration)
    fmt.Printf("轉換為秒: %.0f 秒\n", customDuration.Seconds())
    fmt.Printf("轉換為分鐘: %.2f 分鐘\n", customDuration.Minutes())
    fmt.Printf("轉換為小時: %.2f 小時\n", customDuration.Hours())
    
    // 時間計算
    fmt.Println("\n時間計算:")
    now := time.Now()
    
    // 加減時間
    future := now.Add(2 * time.Hour)
    past := now.Add(-30 * time.Minute)
    
    fmt.Printf("現在: %v\n", now.Format("2006-01-02 15:04:05"))
    fmt.Printf("2小時後: %v\n", future.Format("2006-01-02 15:04:05"))
    fmt.Printf("30分鐘前: %v\n", past.Format("2006-01-02 15:04:05"))
    
    // 計算時間差
    diff := future.Sub(past)
    fmt.Printf("時間差: %v\n", diff)
    
    // 計算到今天結束還有多少時間
    tomorrow := time.Date(now.Year(), now.Month(), now.Day()+1, 0, 0, 0, 0, now.Location())
    untilMidnight := tomorrow.Sub(now)
    fmt.Printf("距離明天零點還有: %v\n", untilMidnight)
}

func main() {
    durationExample()
}

In [ ]:
package main

import (
    "fmt"
    "time"
)

/* 時間格式化和解析 */
func timeFormatExample() {
    now := time.Now()
    
    // Go 的參考時間: Mon Jan 2 15:04:05 MST 2006
    // 記憶法: 01/02 03:04:05PM '06 -0700
    
    fmt.Println("時間格式化:")
    fmt.Printf("預設格式: %v\n", now)
    fmt.Printf("RFC3339: %v\n", now.Format(time.RFC3339))
    fmt.Printf("自訂格式1: %v\n", now.Format("2006-01-02 15:04:05"))
    fmt.Printf("自訂格式2: %v\n", now.Format("2006/01/02 03:04:05 PM"))
    fmt.Printf("中文格式: %v\n", now.Format("2006年01月02日 15時04分05秒"))
    fmt.Printf("星期: %v\n", now.Format("Monday, January 2, 2006"))
    
    // 時間解析
    fmt.Println("\n時間解析:")
    
    timeStrings := []string{
        "2023-12-25 15:30:00",
        "2023/12/25 03:30:00 PM",
        "Dec 25, 2023 3:30 PM",
    }
    
    layouts := []string{
        "2006-01-02 15:04:05",
        "2006/01/02 03:04:05 PM",
        "Jan 2, 2006 3:04 PM",
    }
    
    for i, timeStr := range timeStrings {
        parsedTime, err := time.Parse(layouts[i], timeStr)
        if err != nil {
            fmt.Printf("解析錯誤 %q: %v\n", timeStr, err)
            continue
        }
        fmt.Printf("解析 %q -> %v\n", timeStr, parsedTime)
    }
    
    // 解析帶時區的時間
    fmt.Println("\n時區解析:")
    timeWithZone := "2023-12-25T15:30:00+08:00"
    parsedWithZone, err := time.Parse(time.RFC3339, timeWithZone)
    if err != nil {
        fmt.Printf("時區解析錯誤: %v\n", err)
    } else {
        fmt.Printf("帶時區的時間: %v\n", parsedWithZone)
        fmt.Printf("UTC 時間: %v\n", parsedWithZone.UTC())
    }
}

func main() {
    timeFormatExample()
}

In [ ]:
package main

import (
    "fmt"
    "time"
)

/* 時區處理 */
func timezoneExample() {
    // 獲取不同時區
    utc := time.UTC
    local := time.Local
    
    // 載入特定時區
    tokyo, err := time.LoadLocation("Asia/Tokyo")
    if err != nil {
        fmt.Printf("載入東京時區錯誤: %v\n", err)
        return
    }
    
    newYork, err := time.LoadLocation("America/New_York")
    if err != nil {
        fmt.Printf("載入紐約時區錯誤: %v\n", err)
        return
    }
    
    // 建立同一時刻在不同時區的時間
    baseTime := time.Date(2023, 12, 25, 12, 0, 0, 0, utc)
    
    fmt.Println("同一時刻在不同時區:")
    fmt.Printf("UTC:      %v\n", baseTime)
    fmt.Printf("本地:     %v\n", baseTime.In(local))
    fmt.Printf("東京:     %v\n", baseTime.In(tokyo))
    fmt.Printf("紐約:     %v\n", baseTime.In(newYork))
    
    // 在特定時區建立時間
    fmt.Println("\n在特定時區建立時間:")
    tokyoTime := time.Date(2023, 12, 25, 21, 0, 0, 0, tokyo)
    fmt.Printf("東京時間 21:00: %v\n", tokyoTime)
    fmt.Printf("對應 UTC 時間: %v\n", tokyoTime.UTC())
    fmt.Printf("對應紐約時間: %v\n", tokyoTime.In(newYork))
    
    // 時區資訊
    fmt.Println("\n時區資訊:")
    name, offset := tokyoTime.Zone()
    fmt.Printf("東京時區名稱: %s, 偏移秒數: %d\n", name, offset)
    fmt.Printf("東京時區偏移小時: %.1f\n", float64(offset)/3600)
}

func main() {
    timezoneExample()
}

### 單調時間

單調時間（Monotonic Time）是指保證單調遞增的時間，不受系統時鐘調整影響。Go 的 `time.Now()` 包含了單調時鐘讀數，這對於測量時間間隔特別重要。

### 特點：
- 不會因為系統時鐘調整而跳躍
- 適合測量持續時間和超時
- `time.Since()` 和 `time.Until()` 使用單調時間
- 格式化時單調時鐘讀數會被忽略

### 使用場景：
- 性能測量
- 超時檢查
- 時間間隔計算
- 定時器和計時器

In [ ]:
package main

import (
    "fmt"
    "time"
)

/* 單調時間的使用 */
func monotonicTimeExample() {
    fmt.Println("單調時間示例:")
    
    // 記錄開始時間（包含單調時鐘讀數）
    start := time.Now()
    fmt.Printf("開始時間: %v\n", start)
    
    // 模擬一些工作
    fmt.Println("執行一些工作...")
    time.Sleep(500 * time.Millisecond)
    
    // 計算經過的時間（使用單調時間）
    elapsed := time.Since(start)
    fmt.Printf("經過時間: %v\n", elapsed)
    
    // 手動計算時間差（也使用單調時間）
    end := time.Now()
    manualElapsed := end.Sub(start)
    fmt.Printf("手動計算經過時間: %v\n", manualElapsed)
    
    // 檢查時間是否包含單調時鐘讀數
    fmt.Printf("\n檢查單調時鐘:")
    fmt.Printf("開始時間字串表示: %s\n", start.String())
    
    // 去除單調時鐘讀數
    wallStart := start.Round(0)
    fmt.Printf("去除單調時鐘後: %s\n", wallStart.String())
}

func main() {
    monotonicTimeExample()
}

In [ ]:
package main

import (
    "fmt"
    "time"
)

/* 性能測量使用單調時間 */
func performanceMeasurement() {
    fmt.Println("性能測量示例:")
    
    // 測量函式執行時間
    measureTime := func(name string, fn func()) {
        start := time.Now()
        fn()
        duration := time.Since(start)
        fmt.Printf("%s 執行時間: %v\n", name, duration)
    }
    
    // 測量不同操作的時間
    measureTime("短暫休眠", func() {
        time.Sleep(100 * time.Millisecond)
    })
    
    measureTime("計算操作", func() {
        sum := 0
        for i := 0; i < 1000000; i++ {
            sum += i
        }
    })
    
    measureTime("字串拼接", func() {
        result := ""
        for i := 0; i < 1000; i++ {
            result += "a"
        }
    })
    
    // 比較不同方法的性能
    fmt.Println("\n比較性能:")
    
    const iterations = 5
    var totalDuration time.Duration
    
    for i := 0; i < iterations; i++ {
        start := time.Now()
        time.Sleep(50 * time.Millisecond)
        duration := time.Since(start)
        totalDuration += duration
        fmt.Printf("第 %d 次: %v\n", i+1, duration)
    }
    
    avgDuration := totalDuration / iterations
    fmt.Printf("平均執行時間: %v\n", avgDuration)
}

func main() {
    performanceMeasurement()
}

### 計時器與通時

Go 提供了兩種計時機制：`Timer` 和 `Ticker`。它們都是基於 channel 的，可以很好地與 goroutines 和 select 語句配合使用。

### Timer (計時器)：
- 在指定時間後觸發一次
- 可以被停止或重設
- 適用於超時控制和延遲執行

### Ticker (定時器)：
- 在指定間隔重複觸發
- 可以被停止
- 適用於週期性任務

這兩種機制都提供了非阻塞的方式來處理時間相關的操作。

In [ ]:
package main

import (
    "fmt"
    "time"
)

/* Timer 基本使用 */
func timerExample() {
    fmt.Println("Timer 示例:")
    
    // 建立一個 2 秒後觸發的計時器
    timer := time.NewTimer(2 * time.Second)
    
    fmt.Println("等待計時器觸發...")
    start := time.Now()
    
    // 等待計時器觸發
    <-timer.C
    
    elapsed := time.Since(start)
    fmt.Printf("計時器觸發了，經過時間: %v\n", elapsed)
    
    // 使用 time.After 的簡便方法
    fmt.Println("\n使用 time.After:")
    start2 := time.Now()
    
    select {
    case <-time.After(1 * time.Second):
        elapsed2 := time.Since(start2)
        fmt.Printf("time.After 觸發，經過時間: %v\n", elapsed2)
    }
}

func main() {
    timerExample()
}

In [ ]:
package main

import (
    "fmt"
    "time"
)

/* Timer 的停止和重設 */
func timerControlExample() {
    fmt.Println("Timer 控制示例:")
    
    // 建立計時器
    timer := time.NewTimer(3 * time.Second)
    
    // 在另一個 goroutine 中停止計時器
    go func() {
        time.Sleep(1 * time.Second)
        fmt.Println("停止計時器")
        stopped := timer.Stop()
        fmt.Printf("計時器已停止: %t\n", stopped)
    }()
    
    // 等待計時器或檢查是否已停止
    select {
    case <-timer.C:
        fmt.Println("計時器觸發了")
    case <-time.After(2 * time.Second):
        fmt.Println("等待超時，計時器可能已被停止")
    }
    
    // 重設計時器
    fmt.Println("\n重設計時器:")
    timer.Reset(1 * time.Second)
    
    start := time.Now()
    <-timer.C
    elapsed := time.Since(start)
    fmt.Printf("重設後的計時器觸發，經過時間: %v\n", elapsed)
}

func main() {
    timerControlExample()
}

In [ ]:
package main

import (
    "fmt"
    "time"
)

/* Ticker 基本使用 */
func tickerExample() {
    fmt.Println("Ticker 示例:")
    
    // 建立每 500 毫秒觸發一次的定時器
    ticker := time.NewTicker(500 * time.Millisecond)
    defer ticker.Stop() // 確保停止 ticker 以避免洩漏
    
    start := time.Now()
    count := 0
    
    for {
        select {
        case tickTime := <-ticker.C:
            count++
            elapsed := tickTime.Sub(start)
            fmt.Printf("第 %d 次觸發，經過時間: %v\n", count, elapsed)
            
            // 觸發 5 次後停止
            if count >= 5 {
                fmt.Println("停止 ticker")
                return
            }
        }
    }
}

func main() {
    tickerExample()
}

In [ ]:
package main

import (
    "fmt"
    "time"
)

/* 結合 Timer 和 Ticker 的實際應用 */
func timerTickerApplication() {
    fmt.Println("Timer 和 Ticker 結合應用:")
    
    // 建立一個工作 ticker（每秒執行一次）
    workTicker := time.NewTicker(1 * time.Second)
    defer workTicker.Stop()
    
    // 建立一個超時 timer（5秒後停止）
    timeout := time.NewTimer(5 * time.Second)
    defer timeout.Stop()
    
    // 建立一個狀態報告 ticker（2秒報告一次）
    statusTicker := time.NewTicker(2 * time.Second)
    defer statusTicker.Stop()
    
    workCount := 0
    start := time.Now()
    
    fmt.Println("開始工作，將在 5 秒後停止...")
    
    for {
        select {
        case <-workTicker.C:
            workCount++
            fmt.Printf("執行工作 #%d\n", workCount)
            
        case <-statusTicker.C:
            elapsed := time.Since(start)
            fmt.Printf("[狀態] 運行時間: %v, 完成工作: %d 個\n", 
                elapsed.Round(time.Second), workCount)
            
        case <-timeout.C:
            elapsed := time.Since(start)
            fmt.Printf("\n[超時] 工作結束！總運行時間: %v, 總共完成 %d 個工作\n", 
                elapsed.Round(time.Second), workCount)
            return
        }
    }
}

func main() {
    timerTickerApplication()
}

In [ ]:
package main

import (
    "fmt"
    "time"
)

/* 實現簡單的超時重試機制 */
func timeoutRetryExample() {
    fmt.Println("超時重試機制示例:")
    
    // 模擬一個可能失敗的操作
    simulateWork := func(id int, shouldFail bool) error {
        fmt.Printf("  嘗試 #%d: 開始工作\n", id)
        time.Sleep(800 * time.Millisecond) // 模擬工作時間
        
        if shouldFail {
            return fmt.Errorf("工作失敗")
        }
        fmt.Printf("  嘗試 #%d: 工作成功\n", id)
        return nil
    }
    
    // 帶超時的重試函式
    retryWithTimeout := func(maxRetries int, timeout time.Duration, work func(int, bool) error) error {
        for i := 1; i <= maxRetries; i++ {
            // 為每次嘗試設定超時
            timer := time.NewTimer(timeout)
            done := make(chan error, 1)
            
            // 在 goroutine 中執行工作
            go func(attempt int) {
                // 前兩次嘗試失敗，第三次成功
                shouldFail := attempt < 3
                done <- work(attempt, shouldFail)
            }(i)
            
            // 等待工作完成或超時
            select {
            case err := <-done:
                timer.Stop()
                if err == nil {
                    return nil // 成功
                }
                fmt.Printf("  嘗試 #%d 失敗: %v\n", i, err)
                
            case <-timer.C:
                fmt.Printf("  嘗試 #%d 超時\n", i)
            }
            
            // 如果不是最後一次嘗試，等待一段時間再重試
            if i < maxRetries {
                fmt.Printf("  等待 1 秒後重試...\n")
                time.Sleep(1 * time.Second)
            }
        }
        
        return fmt.Errorf("所有重試都失敗了")
    }
    
    // 執行帶超時的重試
    start := time.Now()
    err := retryWithTimeout(3, 2*time.Second, simulateWork)
    elapsed := time.Since(start)
    
    if err != nil {
        fmt.Printf("\n最終結果: 失敗 - %v\n", err)
    } else {
        fmt.Printf("\n最終結果: 成功\n")
    }
    fmt.Printf("總耗時: %v\n", elapsed.Round(100*time.Millisecond))
}

func main() {
    timeoutRetryExample()
}